In [1]:
!python --version

Python 3.7.15


In [2]:
!wget -O mini.sh https://repo.anaconda.com/miniconda/Miniconda3-py38_4.8.2-Linux-x86_64.sh
!chmod +x mini.sh
!bash ./mini.sh -b -f -p /usr/local
!conda install -q -y jupyter
!conda install -q -y google-colab -c conda-forge
!python -m ipykernel install --name "py38" --user

--2022-11-24 16:12:29--  https://repo.anaconda.com/miniconda/Miniconda3-py38_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89817099 (86M) [application/x-sh]
Saving to: ‘mini.sh’

mini.sh             100%[===================>]  85.66M  52.8MB/s    in 1.6s    

2022-11-24 16:12:31 (52.8 MB/s) - ‘mini.sh’ saved [89817099/89817099]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py38_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py38_0
    - cffi==1.14.0=py38h2e261b9_0
    - chardet==3.0.4=py38_1003
    - conda-package-handling==1.6.0=py38h7b6447c_0
    - conda==4.8.2=py38_0
    - cryptography==2.8

In [3]:
!python --version

Python 3.8.13


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd /content/drive/MyDrive/bert4rec_project/

/content/drive/MyDrive/bert4rec_project


In [6]:
%ls

bert4rec/           data/    Makefile    README.md    requirements.txt  tox.ini
bert4rec.egg-info/  LICENSE  notebooks/  references/  setup.py


In [7]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/drive/MyDrive/bert4rec_project (from -r requirements.txt (line 2))
     |████████████████████████████████| 214 kB 4.5 MB/s 
     |████████████████████████████████| 57 kB 6.8 MB/s 
     |████████████████████████████████| 1.5 MB 53.8 MB/s 
     |████████████████████████████████| 316 kB 90.2 MB/s 
     |████████████████████████████████| 496.0 MB 8.0 kB/s 
     |████████████████████████████████| 17.1 MB 78.6 MB/s 
     |████████████████████████████████| 12.2 MB 94.2 MB/s 
     |████████████████████████████████| 346 kB 94.4 MB/s 
     |████████████████████████████████| 91 kB 13.0 MB/s 
     |████████████████████████████████| 78 kB 9.5 MB/s 
     |████████████████████████████████| 62 kB 11 kB/s 
     |████████████████████████████████| 41 kB 307 kB/s 
     |████████████████████████████████| 96 kB 7.3 MB/s 
     |████████████████████████████████| 81 kB 12.4 MB/s 
     |█

In [8]:
%load_ext autoreload
%autoreload 2

In [9]:
import glob
import pandas as pd
import tensorflow as tf

from dataclasses import dataclass, asdict

## Load parameters

In [10]:
@dataclass
class Config:
    num_layers:int=2
    num_heads:int=8
    d_model:int=128
    dropout:float=0.1
    layer_norm_eps:float=1e-12
    activation:str="gelu"
    vocab_size:int=40857
    max_seq_len:int=256
    learning_rate:float=1e-4
    batch_size:float=64

config=Config()

# Prepare data

In [11]:
from bert4rec.dataprocessor import DataLoader

In [12]:
%cd /content/drive/MyDrive/bert4rec_project/notebooks/

/content/drive/MyDrive/bert4rec_project/notebooks


In [13]:
filenames = glob.glob('../data/processed/*')
filenames

['../data/processed/training.tfrecords',
 '../data/processed/vocab.pkl',
 '../data/processed/test.tfrecords']

In [14]:
train_dataloader = DataLoader(
    input_file_path=filenames[0],
    batch_size=config.batch_size,
    max_seq_length=config.max_seq_len,
)

train_data = train_dataloader.to_tf_dataset()

In [15]:
test_dataloader = DataLoader(
    input_file_path=filenames[2],
    batch_size=config.batch_size,
    max_seq_length=config.max_seq_len,
)

test_data = train_dataloader.to_tf_dataset()

## Train model

In [16]:
from bert4rec.bert import BertModel
from bert4rec.trainer import BertTrainer

In [17]:
model = BertModel(
    num_layers=config.num_layers,
    num_heads=config.num_heads,
    d_model=config.d_model,
    dropout=config.dropout,
    layer_norm_eps=config.layer_norm_eps,
    activation=config.activation,
    vocab_size=config.vocab_size,
    max_seq_len=config.max_seq_len,
)
model.build((config.batch_size, config.max_seq_len))
model.summary()

Model: "bert_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 positional_embedding (Posit  multiple                 5262592   
 ionalEmbedding)                                                 
                                                                 
 transformer_encoder_layer (  multiple                 198528    
 TransformerEncoderLayer)                                        
                                                                 
 transformer_encoder_layer_1  multiple                 198528    
  (TransformerEncoderLayer)                                      
                                                                 
 dropout_2 (Dropout)         multiple                  0         
                                                                 
 dense_4 (Dense)             multiple                  16512     
                                                        

In [18]:
bert_trainer = BertTrainer(model)
bert_trainer.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
)

In [21]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [22]:
bert_trainer.fit(train_data, batch_size=config.batch_size, epochs=5)

Epoch 1/5
2142/2142 [==============================] - 189s 88ms/step - loss: 0.2990 - accuracy: 0.8543
Epoch 2/5
2142/2142 [==============================] - 188s 88ms/step - loss: 0.1531 - accuracy: 0.8545
Epoch 3/5
2142/2142 [==============================] - 189s 88ms/step - loss: 0.1460 - accuracy: 0.8547
Epoch 4/5
2142/2142 [==============================] - 189s 88ms/step - loss: 0.1409 - accuracy: 0.8549
Epoch 5/5
2142/2142 [==============================] - 189s 88ms/step - loss: 0.1370 - accuracy: 0.8551


In [ ]:
tf.saved_model.save(bert_trainer, "../models/bert4rec_movies")